In [1]:
import pandas as pd

In [34]:
df = pd.read_csv("history.csv")

/var/folders/91/thyh1l157sg66sz8yx_40ld00000gn/T/ipykernel_3450/3951251560.py:1: DtypeWarning: Columns (87) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("history.csv")


In [39]:
df = df.iloc[:, -27:]
df.head()

,is_broker_account,is_inventory_account,is_gl_account,is_control_account,is_extract_eligible,last_maintenance_time,last_maintenance_user,is_pledged,special_fee_code,non_calendar_year_end,...,is_hrdc_resp,is_plan_grandfathered,resp_specimen_plan,inserted_at,updated_at,is_olob,retail_last_maintenance_time,retail_last_maintenance_user,visible_in_reports,label
0,f,f,f,f,t,1639612800000000000,T80,NaN,NaN,NaN,...,NaN,NaN,NaN,2023-05-30 14:20:18.531115+00,2023-08-08 18:53:01.439561+00,f,NaN,NaN,f,Churn
1,f,f,f,f,t,1672704000000000000,BATCH,f,NaN,0000,...,f,f,NaN,2023-05-30 14:20:18.531115+00,2023-08-10 21:47:25.370403+00,f,2023-01-03 00:00:00,BATCH,t,Churn
2,f,f,f,f,t,1615766400000000000,GUERINO,f,NaN,00/0,...,f,f,NaN,2023-05-30 14:20:18.531115+00,2023-08-10 21:47:25.370403+00,f,2018-07-17 00:00:00,GUERINO,f,Churn
3,f,f,f,f,t,1609718400000000000,T80,NaN,NaN,NaN,...,NaN,NaN,NaN,2023-05-30 14:20:18.531115+00,2023-08-08 18:53:01.439561+00,f,NaN,NaN,t,Churn
4,f,f,f,f,t,1687824000000000000,H01,NaN,NaN,NaN,...,NaN,NaN,NaN,2023-05-30 14:20:18.531115+00,2023-08-08 20:52:08.352646+00,f,NaN,NaN,f,Churn


In [4]:
def drop_cols(df, cols):
    df = df.drop(cols, axis=1)
    return df

In [5]:
def boolean_map(df, cols):
    for col in cols:
        df[col] = df[col].fillna('f').map({'t': 1, 'f': 0})
    return df

In [31]:
# Times: Convert to number, standardize, fill blanks = 0
def parse_time(df, cols):
    for col in cols:
        df[col] = pd.to_datetime(df[col], errors='coerce').astype(int)

        min = df[col].min()
        max = df[col].max()
        df[col] = (df[col] - min) / (max - min)

        df[col] = df[col].fillna(0)
    return df

In [7]:
# 1. Drop Useless
drop_columns = ['is_broker_account', 'is_gl_account', 'is_control_account', 'is_plan_grandfathered']
df = drop_cols(df, drop_columns)

In [15]:
# 2. Map Booleans
bool_columns = ['is_inventory_account', 'is_extract_eligible', 'is_pledged', 'is_resp', 'is_family_resp', 'is_hrdc_resp']
df = boolean_map(df, bool_columns)

bool_maybe_columns = ['is_olob', 'visible_in_reports', 'use_original_date_for_payment_calc']
df = boolean_map(df, bool_maybe_columns)

In [32]:
# 3. Parse Times
time_columns = ['last_maintenance_time', 'plan_effective_date', 'plan_end_date', 'rrif_original_date', 'inserted_at', 'updated_at', 'retail_last_maintenance_time']

df = parse_time(df, time_columns)

In [10]:
# 4. Miscellaneous/Unsure
df.number_of_beneficiaries.fillna(0, inplace=True)
df.label = df.label.map({'No Churn': 0, 'Churn': 1})

Looking for guidance on these columns
1. last_maintenance_user: A LOT of different users. Map to numbers?
2. retail_last_maintenance_user: Same as above
3. special_fee_code: numbers, characters, blanks
4. resp_specimen plan: numbers (like ids) and blanks

In [38]:
df.head()

,id,type_code,is_registered,country_code,currency_code,is_active,class_id,debit_code,last_trade_date,contract_type,...,is_hrdc_resp,is_plan_grandfathered,resp_specimen_plan,inserted_at,updated_at,is_olob,retail_last_maintenance_time,retail_last_maintenance_user,visible_in_reports,label
0,893724,CASH SWEEP,f,NaN,CAD,t,3.0,0,NaN,NaN,...,NaN,NaN,NaN,2023-05-30 14:20:18.531115+00,2023-08-08 18:53:01.439561+00,f,NaN,NaN,f,Churn
1,1268094,RRIF,t,NaN,CAD,t,5.0,C,2022-07-04,18.0,...,f,f,NaN,2023-05-30 14:20:18.531115+00,2023-08-10 21:47:25.370403+00,f,2023-01-03 00:00:00,BATCH,t,Churn
2,606613,SPOUSAL RRSP,t,NaN,CAD,t,5.0,A,2018-07-05,16.0,...,f,f,NaN,2023-05-30 14:20:18.531115+00,2023-08-10 21:47:25.370403+00,f,2018-07-17 00:00:00,GUERINO,f,Churn
3,741930,CASH,f,NaN,CAD,t,3.0,T,NaN,12.0,...,NaN,NaN,NaN,2023-05-30 14:20:18.531115+00,2023-08-08 18:53:01.439561+00,f,NaN,NaN,t,Churn
4,1137922,CASH,f,NaN,CAD,t,3.0,T,NaN,17.0,...,NaN,NaN,NaN,2023-05-30 14:20:18.531115+00,2023-08-08 20:52:08.352646+00,f,NaN,NaN,f,Churn


In [41]:
df.last_maintenance_time = pd.to_datetime(df.last_maintenance_time)
x = df.last_maintenance_time.isna()
x

0         False
1         False
2         False
3         False
4         False
          ...  
673334    False
673335    False
673336    False
673337    False
673338    False
Name: last_maintenance_time, Length: 673339, dtype: bool

In [37]:
min = df.last_maintenance_time.min()
max = df.last_maintenance_time.max()
min

-9223372036854775808

In [ ]:
min = df[col].min()
max = df[col].max()
df[col] = (df[col] - min) / (max - min)

df[col] = df[col].fillna(0)